In [2]:
import h5py
import numpy as np

f1 = h5py.File("mini_VG-SGG.h5", "r")

In [3]:
list(f1.keys())

['active_object_mask',
 'boxes_1024',
 'boxes_512',
 'img_to_first_box',
 'img_to_first_rel',
 'img_to_last_box',
 'img_to_last_rel',
 'labels',
 'predicates',
 'relationships',
 'split']

In [3]:
import json

df = json.load(open("mini_VG-SGG-dicts.json"))
idx_to_labels = df["idx_to_label"]
idx_to_predicates = df["idx_to_predicate"]

In [8]:
n = len(f1["relationships"])

In [9]:
objects_to_var = {}
rel_to_var = {}

In [10]:
for i in range(n):
    sub, obj, rel = (
        f1["relationships"][i][0],
        f1["relationships"][i][1],
        f1["predicates"][i],
    )
    dic[
        (
            idx_to_labels[str(f1["labels"][sub][0])].upper(),
            idx_to_predicates[str(rel[0])].upper().replace(" ", "_"),
        )
    ].add(idx_to_labels[str(f1["labels"][obj][0])].upper())

In [19]:
clauses = []

for k in dic:
    tail = ""
    for i, p in enumerate(list(dic[k])):
        if i == 0:
            tail += f"{p}(v_2)"
        else:
            tail += f" | {p}(v_2)"
    tail = f"{tail}"
    logic = f"({k[0]}(v_1) & {k[1]}(v_1, v_2)) ==> ({tail})"
    clauses.append(au.expr(logic))

In [1]:
import ltn
import torch

In [2]:
# we define connectives, quantifiers, and SatAgg
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()